In [13]:
import openpyxl
from collections import defaultdict
import random
import matplotlib.pyplot as plt
import time
import csv

In [14]:
def smart_order_router(excelasks, excelbids, side, symbol, qty, order_type='market', price=None):
    needed_qty = qty
    route = {}
    available_qty = 0
    if side == 'buy':
        ask_shared = sorted(excelasks[symbol].items())
        for shared in ask_shared:
            if qty <= 0:
                break
            ask_price = shared[0]
            order_info = shared[1]
            if order_type == 'market' or ask_price <= price:
                for order in order_info:
                    exchanger = order[0]
                    ask_qty = order[1]
                    if exchanger not in route:
                        route[exchanger] = {}
                    if ask_qty >= qty and qty>0:
                        route[exchanger][ask_price] = qty
                        available_qty += qty
                        qty = 0
                    elif qty > 0:
                        route[exchanger][ask_price] = ask_qty
                        available_qty += ask_qty
                        qty = qty - ask_qty
    elif side == 'sell':
        bid_shared = sorted(excelbids[symbol].items(), reverse = True)
        for shared in bid_shared:
            if qty <= 0:
                break
            bid_price = shared[0]
            order_info = shared[1]
            if order_type == 'market' or bid_price >= price:
                for order in order_info:
                    exchanger = order[0]
                    bid_qty = order[1]
                    if exchanger not in route:
                        route[exchanger] = {}
                    if bid_qty >= qty and qty>0:
                        route[exchanger][bid_price] = qty
                        available_qty += qty
                        qty = 0
                    elif qty > 0:
                        route[exchanger][bid_price] = bid_qty
                        available_qty += bid_qty
                        qty = qty - bid_qty
    order = {'route':route, 'leave_qty':needed_qty - available_qty}
    return order    

In [15]:
def test_order_number():
	excelbids = defaultdict(lambda:defaultdict(list))
	excelasks = defaultdict(lambda:defaultdict(list))
	for i in range(0,50):
		exchanger_name = "Exchanger"+str(i)
		for _ in range(0, 5):
			excelbids['Microsofts'][round(random.uniform(183.5, 184.5), 2)].append([exchanger_name,random.randint(100, 500)])
			excelasks['Microsofts'][round(random.uniform(183.5, 184.5), 2)].append([exchanger_name,random.randint(100, 500)])
	results = []
	for i in range(1, 21, 1):
		order_counts = i*100
		start_time = time.time()
		for order_count in range(0, order_counts):
			qty = random.randint(1000, 5000)
			limit_price = round(random.uniform(183.5, 184.5), 2)
			side = random.choice(['buy', 'sell'])
			route = smart_order_router(excelasks, excelbids, 'sell', 'Microsofts', qty, 'limit', limit_price)
		end_time = time.time()
		results.append([i, end_time - start_time])
	with open('./greedy_order.csv', 'w') as f:
		writer = csv.writer(f)
		for result in results:
			writer.writerow(result)

In [16]:
test_order_number()

In [17]:
def test_exchanger_number():
    results = []
    for i in range(0, 51, 5):
        exchanger_num = i
        if exchanger_num == 0:
            exchanger_num = 2
        excelbids = defaultdict(lambda:defaultdict(list))
        excelasks = defaultdict(lambda:defaultdict(list))
        for i in range(0, exchanger_num):
            exchanger_name = "Exchanger"+str(i)
            for _ in range(0, 5):
                excelbids['Microsofts'][round(random.uniform(183.5, 184.5), 2)].append([exchanger_name,random.randint(100, 500)])
                excelasks['Microsofts'][round(random.uniform(183.5, 184.5), 2)].append([exchanger_name,random.randint(100, 500)])
        start_time = time.time()
        for order_count in range(0, 2000):
            qty = random.randint(1000, 5000)
            limit_price = round(random.uniform(183.5, 184.5), 2)
            side = random.choice(['buy', 'sell'])
            route = smart_order_router(excelasks, excelbids, 'sell', 'Microsofts', qty, 'limit', limit_price)
        end_time = time.time()
        results.append([exchanger_num, end_time - start_time])
    with open('./greedy_exchanger_50.csv', 'w') as f:
        writer = csv.writer(f)
        for result in results:
            writer.writerow(result)

In [18]:
test_exchanger_number()